In [19]:
import pandas as pd

bible = pd.read_csv('t_bbe.csv')
key = pd.read_csv('key_english.csv')

In [2]:
bible.shape

(31103, 5)

In [3]:
bible.head()

,id,b,c,v,t
0,1001001,1,1,1,At the first God made the heaven and the earth.
1,1001002,1,1,2,And the earth was waste and without form; and ...
2,1001003,1,1,3,"And God said, Let there be light: and there wa..."
3,1001004,1,1,4,"And God, looking on the light, saw that it was..."
4,1001005,1,1,5,"Naming the light, Day, and the dark, Night. An..."


In [4]:
bible['b'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66])

In [20]:
bible=bible.merge(key[['b','n']], on='b')


In [21]:
bible

,id,b,c,v,t,n
0,1001001,1,1,1,At the first God made the heaven and the earth.,Genesis
1,1001002,1,1,2,And the earth was waste and without form; and ...,Genesis
2,1001003,1,1,3,"And God said, Let there be light: and there wa...",Genesis
3,1001004,1,1,4,"And God, looking on the light, saw that it was...",Genesis
4,1001005,1,1,5,"Naming the light, Day, and the dark, Night. An...",Genesis
...,...,...,...,...,...,...
31098,66022017,66,22,17,"And the Spirit and the bride say, Come. And le...",Revelation
31099,66022018,66,22,18,For I say to every man to whose ears have come...,Revelation
31100,66022019,66,22,19,And if any man takes away from the words of th...,Revelation
31101,66022020,66,22,20,"He who gives witness to these things says, Tru...",Revelation


In [12]:
bible_by_book = bible.groupby(['b','n'])['t'].apply(' '.join).reset_index()

In [37]:
bible_by_chap = bible.groupby(['b','n','c'])['t'].apply(' '.join).reset_index()

In [38]:
len(bible_by_chap['c'])

1189

In [26]:
bible_by_chap.assign(max_chap = bible_by_chap.apply(lambda x:(x.groupby('n')['c'].agg('max')),axis=1))

AttributeError: 'str' object has no attribute 'ndim'

In [45]:
max_chap = bible_by_chap.groupby(['n']).agg({'c':max}).rename(columns = {'c':'max_chap'}).reset_index()

In [46]:
bible_by_chap=bible_by_chap.merge(max_chap, on = 'n')

In [69]:
bible_by_chap.groupby(['n','c']).agg({'t':len})

t
n            c    
1 Chronicles 1   1
             2   1
             3   1
             4   1
             5   1
...             ..
Zechariah    13  1
             14  1
Zephaniah    1   1
             2   1
             3   1

[1189 rows x 1 columns]

In [55]:
bible_by_chap.groupby(['n','c']).agg({'t':len}).reset_index().rename(columns = {'t':'book_len'}).sort_values('book_len')  

,n,book_len
42,Jude,1
16,3 John,1
54,Obadiah,1
55,Philemon,1
10,2 John,1
...,...,...
26,Ezekiel,48
29,Genesis,50
36,Jeremiah,52
34,Isaiah,66


In [61]:
len(bible_by_chap[bible_by_chap['n'] == 'Jude']['t'].values[0])

3902

In [21]:
bible_by_chap['n'][0]

'Genesis'

In [8]:
from transformers import pipeline

summarizer = pipeline('summarization',model = 'facebook/bart-large-cnn')



/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-20 01:49:34.363183: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-20 01:49:34.363224: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
print(summarizer(bible_by_chap[bible_by_chap['b'] == 1]['t'].values[0], max_length=1500, min_length=500, do_sample=False))


Token indices sequence length is longer than the specified maximum sequence length for this model (46319 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self